## Train Ml Model 

In [7]:
!pip install xgboost 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 46.4 MB/s eta 0:00:00


In [8]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Encode target labels (if necessary)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data into DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # Multi-class classification
    'num_class': 3,  # Number of target classes
    'eval_metric': 'mlogloss',
    'max_depth': 3,
    'learning_rate': 0.1,
    'seed': 42
}

# Train the XGBoost model
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

# Make predictions
y_pred = model.predict(dtest)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'XGBoost Model Accuracy: {accuracy:.4f}')


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


XGBoost Model Accuracy: 1.0000


## Saving Model File 

In [9]:
model_file_name = "iris_xgb_model"
model.save_model(model_file_name)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:20:40] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


## Loading Model file

In [10]:
# Load the saved model
loaded_model = xgb.Booster()
loaded_model.load_model(model_file_name)

In [11]:
# Perform inference
y_pred_loaded = loaded_model.predict(dtest)

# Evaluate the loaded model
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print(f'Loaded XGBoost Model Accuracy: {accuracy_loaded:.4f}')

Loaded XGBoost Model Accuracy: 1.0000


### Perform inference on a single row

In [12]:
# Perform inference on a single row
single_row = X_test[0].reshape(1, -1)
dsingle = xgb.DMatrix(single_row)
single_pred = loaded_model.predict(dsingle)
print(f'Single row prediction: {single_pred[0]}')

Single row prediction: 1.0


## Saving Label Encoder

In [13]:
import joblib
# Save the LabelEncoder
label_encoder_file = "iris_label_encoder.pkl"
joblib.dump(label_encoder, label_encoder_file)

['iris_label_encoder.pkl']

## Sagemaker Setup

In [14]:
%%time

import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

bucket = "yourbucket-name"

[01/29/25 16:20:41] INFO     Found credentials from IAM Role:                                   ]8;id=693687;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=21387;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

CPU times: user 809 ms, sys: 43.4 ms, total: 852 ms
Wall time: 513 ms


## Setting bucket path

In [ ]:
prefix = "test_deployment"
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

print("My Bucket Path: ",bucket_path)

## Converting Saved Model to .tar.gz format

In [16]:
!pwd

/home/ec2-user/SageMaker/Shantanu/Deployment


In [17]:
!tar czvf model.tar.gz $model_file_name

iris_xgb_model


## Upload the pre-trained model to S3

In [ ]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

print(f"Model uploaded to S3 at path: s3://{bucket}/{key}")

## Fetch pre-build xgboost image

In [20]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, "xgboost", "1.7-1")   #"1.7-1"   #0.90-2
print(container)

[01/29/25 16:23:27] WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=191990;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=31286;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

[01/29/25 16:23:28] INFO     Ignoring unnecessary instance type: None.                            ]8;id=957863;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=796987;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-xgboost:1.7-1


## Set up hosting for the model

### Import model into hosting
#### This involves creating a SageMaker model from the model file previously uploaded to S3.

In [19]:
import xgboost as xgb
print(xgb.__version__)

2.1.3


In [ ]:
%%time
from time import gmtime, strftime
model_file_name = "test-model"
model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)
sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

## Create endpoint configuration

SageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, you can create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration describes the instance type required for model deployment.

In [ ]:
from time import gmtime, strftime

endpoint_config_name = "testdeployment-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

## Create endpoint

Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [ ]:
%%time
import time

endpoint_name = "testdeployment-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

output

testdeployment-2025-01-29-16-48-46
arn:aws:sagemaker: Removed some part for privacy
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:Removed some part for privacy
Status: InService
CPU times: user 74.6 ms, sys: 9.13 ms, total: 83.7 ms
Wall time: 4min

## Validate Deployed Endpoint

In [29]:
runtime_client = boto3.client("runtime.sagemaker")

In [30]:
%%time
import json

# Single row data to test (customize based on your test data)
single_row = X_test[0].reshape(1, -1)  # Example: Taking the first row from the test set

# Convert the row to CSV format (as a string)
import pandas as pd
single_row_df = pd.DataFrame(single_row)
payload = single_row_df.to_csv(header=False, index=False)

# Perform inference
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

# Read and decode the result
result = response["Body"].read().decode("ascii")
print("Predicted Class Probabilities: {}".format(result))


Predicted Class Probabilities: 1.0

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 130 ms


## Delete endpoint (optional)

In [31]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'd6956c04-8ada-4418-9628-72ab8a84f333',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd6956c04-8ada-4418-9628-72ab8a84f333',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 29 Jan 2025 16:54:39 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}